In [ ]:
pip install opfunu mealpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 32.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from mealpy.swarm_based.SSA import OriginalSSA


In [ ]:
def objective_function(solution):
    selected_features = np.where(solution > 0.5)[0]
    if len(selected_features) == 0:
        return float('inf')  # Prevent selecting no features

    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    model = Sequential()
    model.add(Dense(64, input_dim=X_train_selected.shape[1], activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train_selected, y_train, epochs=10, batch_size=32, verbose=0)

    _, accuracy = model.evaluate(X_test_selected, y_test, verbose=0)
    return 1 - accuracy  # Minimizing the inverse of accuracy


In [ ]:
# Load data
data = pd.read_csv('/content/Sample_Training_1.csv')

# Feature and target separation
X = data.drop('fraudRisk', axis=1).values
y = data['fraudRisk'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from mealpy.utils.space import FloatVar
# Salp Swarm Algorithm
problem = {
    "obj_func": objective_function,
    "lb": [0] * X_train.shape[1],
    "ub": [1] * X_train.shape[1],
    "minmax": "min",
    "verbose": True,
    "bounds": [FloatVar(0, 1) for _ in range(X_train.shape[1])]  # Use FloatVar to define bounds
}

model = OriginalSSA(epoch=30, pop_size=20)
best_position, best_fitness = model.solve(problem)
selected_features = np.where(best_position > 0.5)[0]

print(f"Selected features: {selected_features}")

INFO:mealpy.swarm_based.SSA.OriginalSSA:Solving single objective optimization problem.
INFO:mealpy.swarm_based.SSA.OriginalSSA:>>>Problem: P, Epoch: 1, Current best: 0.04100000858306885, Global best: 0.04100000858306885, Runtime: 392.11489 seconds
INFO:mealpy.swarm_based.SSA.OriginalSSA:>>>Problem: P, Epoch: 2, Current best: 0.04100000858306885, Global best: 0.04100000858306885, Runtime: 406.73063 seconds
INFO:mealpy.swarm_based.SSA.OriginalSSA:>>>Problem: P, Epoch: 3, Current best: 0.04066663980484009, Global best: 0.04066663980484009, Runtime: 406.60648 seconds
INFO:mealpy.swarm_based.SSA.OriginalSSA:>>>Problem: P, Epoch: 4, Current best: 0.040499985218048096, Global best: 0.040499985218048096, Runtime: 385.67235 seconds
INFO:mealpy.swarm_based.SSA.OriginalSSA:>>>Problem: P, Epoch: 5, Current best: 0.040499985218048096, Global best: 0.040499985218048096, Runtime: 410.16945 seconds
INFO:mealpy.swarm_based.SSA.OriginalSSA:>>>Problem: P, Epoch: 6, Current best: 0.040499985218048096, Glo

TypeError: cannot unpack non-iterable Agent object

In [ ]:
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]


In [ ]:
#########################################################################################################################

In [ ]:
pip install tensorflow keras pyswarms pandas scikit-learn


In [ ]:
!pip install pyswarms --upgrade
# Upgrade pyswarms to the latest version

import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pyswarms.single.global_best import GlobalBestPSO # Import the GlobalBestPSO class
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
# Load the dataset
df = pd.read_csv('/content/sample.csv')

# Assuming the last column is the target
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
def fitness_function(solution):
    # Select features based on the solution
    selected_features = np.where(solution > 0.5)[0]  # Get indices of selected features
    if selected_features.size == 0:
        return 1.0  # If no features are selected, return a high fitness value

    # Ensure selected features are within bounds
    selected_features = np.clip(selected_features, 0, X_train.shape[1] - 1)  # Clip indices to be within valid range

    X_train_selected = X_train[:, selected_features]  # Select columns using indices
    X_test_selected = X_test[:, selected_features]

    model = create_model(input_dim=X_train_selected.shape[1])
    model.fit(X_train_selected, y_train, epochs=50, batch_size=32, verbose=0)

    y_pred = (model.predict(X_test_selected) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    return 1.0 - accuracy  # Minimize the inverse of accuracy

In [ ]:
!pip install pyswarms --upgrade
# Upgrade pyswarms to the latest version

import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pyswarms.single.global_best import GlobalBestPSO # Import the GlobalBestPSO class
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
# Import the SSA class from pyswarms
from pyswarms.single import GlobalBestPSO as SSA



In [ ]:
# Define SSA parameters
options = {
    'c1': 1.5, 'c2': 1.5, 'w': 0.5,
    'k': 3, 'p': 2
}

# Initialize SSA
optimizer = SSA(n_particles=30, dimensions=X_train.shape[1], options=options)

# Run SSA
cost, pos = optimizer.optimize(fitness_function, iters=100)

# Get the best feature subset
best_features = np.where(pos > 0.5, 1, 0)

2024-06-27 05:18:42,811 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.5, 'k': 3, 'p': 2}
pyswarms.single.global_best:   0%|          |0/100

2/2 [==============================] - 0s 8ms/step


pyswarms.single.global_best:   1%|          |1/100, best_cost=0.125

2/2 [==============================] - 0s 5ms/step


pyswarms.single.global_best:   2%|▏         |2/100, best_cost=0.1

...

pyswarms.single.global_best:  98%|█████████▊|98/100, best_cost=0.1

2/2 [==============================] - 0s 5ms/step


pyswarms.single.global_best:  99%|█████████▉|99/100, best_cost=0.1

2/2 [==============================] - 0s 7ms/step


pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.1
2024-06-27 05:21:54,990 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.09999999999999998, best pos: [0.29627564 1.12145684 1.01704153 0.71269219 0.53290079 0.83342541
 0.80087487 0.15536424]


In [ ]:
pip install lazypredict

In [ ]:
!pip install SSA

In [ ]:
# import the important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# import the libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv1D,BatchNormalization,Dropout

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout

In [ ]:
credit =pd.read_csv("/content/Sample_Train1.csv")
print(credit.head())

   custID  gender  state  cardholder  balance  numTrans  numIntlTrans  \
0       1       1     35           1     3000         4            14   
1       2       2      2           1        0         9             0   
2       3       2      2           1        0        27             9   
3       4       1     15           1        0        12             0   
4       5       1     46           1        0        11            16   

   creditLine  fraudRisk  
0           2          0  
1          18          0  
2          16          0  
3           5          0  
4           7          0  


In [ ]:
print(credit.isnull().sum())
print(credit['fraudRisk'].value_counts())

custID          0
gender          0
state           0
cardholder      0
balance         0
numTrans        0
numIntlTrans    0
creditLine      0
fraudRisk       0
dtype: int64
fraudRisk
0    9396
1     603
Name: count, dtype: int64


In [ ]:
non_fraud = credit[credit['fraudRisk'] == 0]
fraud = credit[credit['fraudRisk'] == 1]
print(credit['fraudRisk'].value_counts())

fraudRisk
0    9396
1     603
Name: count, dtype: int64


In [ ]:
print(non_fraud.shape, fraud.shape)


(9396, 9) (603, 9)


In [ ]:
credit = pd.concat([fraud, non_fraud], ignore_index=True)
print(credit.head(15))

    custID  gender  state  cardholder  balance  numTrans  numIntlTrans  \
0       30       2     49           1     5192        84             0   
1       44       2     11           1    17656        16             3   
2       57       2     10           1     8000        30            60   
3       64       1     36           1    13876       100             2   
4       66       1     15           1    10144        26             0   
5       87       1     10           1    10681       100             0   
6      111       2     35           1    11000        14             0   
7      156       2     10           1    12213        26             0   
8      160       2     43           1        0       100             7   
9      162       2     49           1     5849       100             0   
10     174       1     48           1     5626       100             0   
11     272       1     22           1    10388        26             0   
12     287       1     20           1 

In [ ]:
# now let us again check the value counts
print(credit.fraudRisk.value_counts())

fraudRisk
0    9396
1     603
Name: count, dtype: int64


In [ ]:
# now dividing the dataframe into dependent and independent varaible
X=credit.drop(['fraudRisk'], axis=1)
y=credit.fraudRisk

# check the shape
print(X.shape, y.shape)

(9999, 8) (9999,)


In [ ]:
class SalpSwarmAlgorithm:
    def __init__(self, obj_func, dim, population_size, max_iter):
        self.obj_func = obj_func
        self.dim = dim
        self.population_size = population_size
        self.max_iter = max_iter

    def initialize_population(self):
        population = np.random.rand(self.population_size, self.dim)
        return population

    def fitness(self, population):
        fitness_values = np.array([self.obj_func(ind) for ind in population])
        return fitness_values

    def update_position(self, population, leader_position):
        for i in range(1, self.population_size):
            for j in range(self.dim):
                c1 = np.random.rand()
                c2 = np.random.rand()
                population[i, j] = c1 * (leader_position[j] - population[i, j]) + c2 * (population[i, j] - leader_position[j])
        return population

    def run(self):
        population = self.initialize_population()
        fitness_values = self.fitness(population)
        leader_position = population[np.argmin(fitness_values)]
        leader_fitness = np.min(fitness_values)

        for t in range(self.max_iter):
            population = self.update_position(population, leader_position)
            fitness_values = self.fitness(population)
            if np.min(fitness_values) < leader_fitness:
                leader_position = population[np.argmin(fitness_values)]
                leader_fitness = np.min(fitness_values)

        return leader_position

# Define the objective function
def objective_function(feature_subset):
    selected_features = np.where(feature_subset > 0.5)[0]
    if len(selected_features) == 0:
        return 1e6  # Penalize empty selection

    X_selected = X.iloc[:, selected_features]
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=99)

    model = Sequential([
        Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(X_train.shape[1], 1)),
        Dropout(0.5),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=5, verbose=0)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return 1 - accuracy


In [ ]:
# Initialize SSA parameters
population_size = 30
max_iter = 100
dim = X.shape[1]

# Apply SSA for feature selection
ssa = SalpSwarmAlgorithm(objective_function, dim, population_size, max_iter)
best_features = ssa.run()

selected_features = np.where(best_features > 0.5)[0]
print("Selected features:", selected_features)



Selected features: []


In [ ]:
# Update the dataset with selected features
X_selected = X.iloc[:, selected_features]

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=99)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data to fit the CNN input requirements
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)


ValueError: at least one array or dtype is required

In [ ]:
# Define the model
model = Sequential()

# Input layer
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(X_train.shape[1], 1)))

# Add hidden layers
for _ in range(9):
    model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
    model.add(Dropout(0.5))

# Flatten the output
model.add(Flatten())

# Fully connected layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


In [ ]:
# Plot the learning curve
def plot_learningcurve(history, epochs):
    epoch = range(1, epochs + 1)
    # accuracy
    plt.plot(epoch, history.history['accuracy'])
    plt.plot(epoch, history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    # loss
    plt.plot(epoch, history.history['loss'])
    plt.plot(epoch, history.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

plot_learningcurve(history, 20)
